In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [2]:
from helpers import load_csv_data,predict_labels

DATA_TRAIN_PATH = 'data/train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


## Prepare cost and prediction functions

In [3]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    # ***************************************************
    cost = y.dot(X.dot(w)) + 0.5 * lambda_ * np.inner(w,w)
    return cost

In [4]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    y_pred = predict_labels(w,X)
    acc    = (np.mean(y_pred == y))
    return acc

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [5]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    def is_support(y_n, x_n, w):
        return y_n * x_n @ w < 1
 
    grad = - y_n * x_n.T if is_support(y_n, x_n, w) else np.zeros_like(x_n.T)
    grad = np.squeeze(grad) + lambda_ * w
    return grad
    


Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [6]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 1
    lambda_ = 0.01
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=1443191301.1294844
iteration=10000, cost=80361643.34320377
iteration=20000, cost=74343681.15637304
iteration=30000, cost=69095372.4172969
iteration=40000, cost=66596494.715882294
iteration=50000, cost=62900563.89568418
iteration=60000, cost=60620387.774703555
iteration=70000, cost=58212981.74372685
iteration=80000, cost=55743000.27946872
iteration=90000, cost=56344400.16970303
training accuracy = 0.6382


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [10]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    g = (y_n * x_n.dot(w) - 1)

    if old_alpha_n == 0:
        g = min(g, 0)
    elif old_alpha_n == 1.0 / lambda_:
        g = max(g, 0)
    else:
        g = g
    if g != 0:
        alpha[n] = min( max(old_alpha_n - g / (x_n.T.dot(x_n)), 0.0),  1.0 / lambda_)

        # compute the corresponding update on the primal vector w
        w += 1.0 * (alpha[n] - old_alpha_n) * y_n * x_n
    return w, alpha

In [11]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    return np.sum(alpha * (1 - y * (X.dot(w))) + lambda_ / 2 * np.sum(w ** 2))


In [12]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 100000
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:1446.80201, dual:0.00000, gap:1446.80201
iteration=10000, primal:3429.65812, dual:0.01854, gap:3429.63958
iteration=20000, primal:981.71449, dual:0.02595, gap:981.68855
iteration=30000, primal:3491.69946, dual:0.03501, gap:3491.66445
iteration=40000, primal:5068.95301, dual:0.04551, gap:5068.90749
iteration=50000, primal:4041.85443, dual:0.05420, gap:4041.80023
iteration=60000, primal:3354.47682, dual:0.06562, gap:3354.41120
iteration=70000, primal:4834.97738, dual:0.07560, gap:4834.90178
iteration=80000, primal:3351.41131, dual:0.08515, gap:3351.32617
iteration=90000, primal:995.17707, dual:0.09625, gap:995.08083
training accuracy = 0.5374
